##**What is CP**?

Constraint Programming (CP) is a powerful paradigm used to solve optimization and decision-making problems by defining a set of constraints and variables and letting a solver find a feasible or optimal solution.

Unlike traditional programming, where we write step-by-step instructions, in CP, we define the problem as a set of rules (constraints), and the solver searches for a valid solution.

###**Problem Statement: Autonomous Car Lane & Speed Optimization**

Autonomous vehicles must navigate safely while following specific rules and constraints such as speed limits, smooth acceleration, lane constraints, and obstacle avoidance. The challenge is to optimize the car's movement while adhering to these constraints.

###***Solution***
This program applies **constraint programming** using **Google OR-Tools' CP-SAT solver** to define and solve the problem. The program optimizes the car's speed and lane position while ensuring smooth transitions and avoiding obstacles.


In [1]:
!pip install ortools

### **Objective**  
We need to optimize an **autonomous car’s speed and lane selection** over a period of **10 time steps** while ensuring:
- Safe driving constraints (speed changes gradually)
- Allowed lane change constraints (no skipping lanes)
- Avoiding obstacles (blocked lanes)
- Encouraging lane diversity (to avoid staying in the same lane too long)
- Maximizing the car’s speed while following the constraints  


### **Decision Variables**  
1. **Speed at each time step**  
   - Range: `30 km/h` (min) to `120 km/h` (max)  
   - Variable: `speed_t[i]` for each time step `i`  
2. **Lane at each time step**  
   - Allowed lanes: `1`, `2`, `3`  
   - Variable: `lane_t[i]` for each time step `i`  



### **Constraints**  
1. **Smooth Acceleration/Deceleration**  
   - The car **cannot change speed by more than 10 km/h per step**.  
2. **Lane Change Restriction**  
   - The car **can only move one lane at a time** (no skipping).  
3. **Obstacle Avoidance**  
   - At `t=5`, **lane 2 is blocked**, so the car must not be in lane 2.  
4. **Encouraging Lane Diversity**  
   - The car **should not stay in the same lane too long**.  
   - It **must change lanes at least once** over the planning horizon.  
5. **Objective Function**  
   - **Maximize total speed** while **penalizing lane stagnation** to encourage lane changes.  

---


---

### **1️⃣ Defining the CP Model**
```python
from ortools.sat.python import cp_model

def autonomous_car_constraints():
    model = cp_model.CpModel()
```
- **OR-Tools Library**: We import Google's **OR-Tools** library, which includes the **Constraint Programming (CP) solver**.
- **CpModel**: We create an instance of the **CpModel** class, which is used to define our optimization problem.

---

### **2️⃣ Defining Variables**
#### **Speed Variable**
```python
max_speed = 120  # km/h
min_speed = 30   # km/h
time_steps = 10  # Planning horizon
speed = [model.NewIntVar(min_speed, max_speed, f'speed_t{i}') for i in range(time_steps)]
```
- **Speed Variables**: We define an array of **speed variables** for each time step. Each variable represents the car's speed, and the speed is constrained between **30 km/h and 120 km/h**.

#### **Lane Variable**
```python
lane_positions = [1, 2, 3]  # Available lanes
lane = [model.NewIntVar(min(lane_positions), max(lane_positions), f'lane_t{i}') for i in range(time_steps)]
```
- **Lane Variables**: We define an array of **lane variables** for each time step. Each variable represents the lane the car occupies, with possible values **1, 2, or 3**.

---

### **3️⃣ Adding Constraints**
#### **1️⃣ Speed Change Constraints (Smooth Acceleration)**
```python
for t in range(1, time_steps):
    model.Add(speed[t] - speed[t - 1] <= 10)  # Increase speed by max 10
    model.Add(speed[t - 1] - speed[t] <= 10)  # Decrease speed by max 10
```
- **Smooth Acceleration and Deceleration**: These constraints ensure that the car can **only change its speed by a maximum of 10 km/h** between two consecutive time steps, allowing for smooth acceleration and deceleration.

#### **2️⃣ Lane Change Constraints (No Skipping Lanes)**
```python
for t in range(1, time_steps):
    lane_diff = model.NewIntVar(0, 1, f'lane_diff_t{t}')
    model.AddAbsEquality(lane_diff, lane[t] - lane[t - 1])
    model.Add(lane_diff <= 1)
```
- **No Skipping Lanes**: This set of constraints ensures that the car **changes lanes gradually**, meaning it cannot skip from one lane to another in one time step. The car can only move by **one lane at a time**.

#### **3️⃣ Obstacle Avoidance**
```python
model.Add(lane[5] != 2)
```
- **Avoid Lane 2 at t=5**: This constraint ensures that the car does not enter **lane 2 at time step 5**, perhaps because it's blocked by an obstacle.

#### **4️⃣ Avoiding Lane Stagnation**
```python
lane_stagnation = [model.NewBoolVar(f'lanes_same_t{t}') for t in range(1, time_steps)]
for t in range(1, time_steps):
    model.Add(lane[t] == lane[t - 1]).OnlyEnforceIf(lane_stagnation[t - 1])
```
- **Penalizing Lane Stagnation**: This set of constraints introduces a Boolean variable `lanes_same_t[t]`, which will be **1 if the car remains in the same lane** at time `t`. It is used later to penalize the car for staying in the same lane too long.

#### **5️⃣ Encouraging Lane Changes**
```python
for t in range(1, time_steps - 2):
    model.Add(lane[t] != lane[t + 2])  # Ensure lane change within 3 steps
```
- **Encouraging Lane Changes**: This constraint forces the car to **change lanes at least once every 3 time steps**. This prevents the car from staying in the same lane for too long.

```python
lane_changes = [model.NewBoolVar(f'lane_change_t{t}') for t in range(1, time_steps)]
for t in range(1, time_steps):
    model.Add(lane[t] != lane[t - 1]).OnlyEnforceIf(lane_changes[t - 1])

model.Add(sum(lane_changes) >= 1)
```
- **At Least One Lane Change**: This constraint ensures that the car **changes lanes at least once** during the entire planning horizon (10 time steps).

---

### **4️⃣ Objective Function**
```python
model.Maximize(
    sum(speed) - 10 * sum(lane_stagnation[t - 1] for t in range(1, time_steps))
)
```
- **Maximizing Speed**: The **objective function** aims to **maximize the total speed** over all time steps.
- **Penalizing Lane Stagnation**: The term `-10 * sum(lane_stagnation)` **penalizes the car for staying in the same lane** too long, encouraging the car to change lanes regularly.

---

### **5️⃣ Solving the Problem**
```python
solver = cp_model.CpSolver()
status = solver.Solve(model)
```
- **Solver**: The **CpSolver** is used to find the best solution that satisfies all constraints. It will either return **OPTIMAL**, **FEASIBLE**, or **INFEASIBLE** based on the results.

#### **Printing the Results**
```python
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for t in range(time_steps):
        print(f'Time {t}: Speed = {solver.Value(speed[t])} km/h, Lane = {solver.Value(lane[t])}')
else:
    print("No feasible solution found.")
```
- If a **valid solution** is found, the program prints the **speed and lane** at each time step.
- If no feasible solution exists, it prints a message indicating that no solution was found.

---



In [14]:
from ortools.sat.python import cp_model

def autonomous_car_constraints():
    model = cp_model.CpModel()

    # Variables
    max_speed = 120  # km/h
    min_speed = 30   # km/h
    lane_positions = [1, 2, 3]  # Available lanes
    time_steps = 10  # Planning horizon

    speed = [model.NewIntVar(min_speed, max_speed, f'speed_t{i}') for i in range(time_steps)]
    lane = [model.NewIntVar(min(lane_positions), max(lane_positions), f'lane_t{i}') for i in range(time_steps)]

    # Constraints
    for t in range(1, time_steps):
        # Speed change constraints (gradual acceleration/deceleration)
        model.Add(speed[t] - speed[t - 1] <= 30)  # Increased flexibility from 10 → 30
        model.Add(speed[t - 1] - speed[t] <= 30)  # Allow greater variation

        # Lane change constraints (only one lane change at a time)
        lane_diff = model.NewIntVar(-1, 1, f'lane_diff_t{t}')
        model.Add(lane_diff == lane[t] - lane[t - 1])

    # **Obstacle avoidance constraint** (Ensure lane 2 is not used at `t=5`)
    model.Add(lane[5] != 2)

    # **Loosen lane change constraints**
    lane_changes = [model.NewBoolVar(f'lane_change_t{t}') for t in range(1, time_steps)]
    for t in range(1, time_steps):
        model.Add(lane[t] != lane[t - 1]).OnlyEnforceIf(lane_changes[t - 1])

    # **Remove unnecessary restrictions on total lane changes**
    model.Add(sum(lane_changes) >= 1)  # At least one lane change
    model.Add(sum(lane_changes) <= 8)  # Allow enough flexibility

    # **Objective function:** Maximize speed while allowing smooth lane changes
    model.Maximize(sum(speed) - 2 * sum(lane_changes))

    # **Solve with debugging logs**
    solver = cp_model.CpSolver()
    solver.parameters.log_search_progress = True  # Enables solver logging
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        for t in range(time_steps):
            print(f'Time {t}: Speed = {solver.Value(speed[t])} km/h, Lane = {solver.Value(lane[t])}')
    else:
        print("No feasible solution found. Adjust constraints.")

if __name__ == "__main__":
    autonomous_car_constraints()

Time 0: Speed = 120 km/h, Lane = 1
Time 1: Speed = 120 km/h, Lane = 2
Time 2: Speed = 120 km/h, Lane = 1
Time 3: Speed = 120 km/h, Lane = 1
Time 4: Speed = 120 km/h, Lane = 2
Time 5: Speed = 120 km/h, Lane = 3
Time 6: Speed = 120 km/h, Lane = 3
Time 7: Speed = 120 km/h, Lane = 2
Time 8: Speed = 120 km/h, Lane = 1
Time 9: Speed = 120 km/h, Lane = 1
